In [1]:
import requests
import polars as pl
import streamlit as st
import altair as alt
from io import StringIO
import plotly
import plotly.express as px

/Users/scottbrydon/Desktop/snow_and_water/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# TODO
1. Find descriptions of metrics to display when that metric is graphed
2. SWE stacked bar chart
3. Historical and current snow
4. Real-time wind streaming metrics
5. Make everything dynamic inputs
6. Display forecasts

In [2]:
duration="HOURLY"
elements = "SNWD%2CSNDN%2CSNRR%2CSWE%2CWTEQ%2CTOBS"
#end_date = datetime.now()
#start_date = end_date - timedelta(days=years_back * 365)

weather_data = requests.get(f'https://wcc.sc.egov.usda.gov/awdbRestApi/services/v1/data?stationTriplets=784%3ACA%3ASNTL&elements={elements}&duration={duration}&beginDate=2025-10-01%2013%3A00&periodRef=END&centralTendencyType=NONE&returnFlags=false&returnOriginalValues=false&returnSuspectData=false')
reference_data = requests.get('https://wcc.sc.egov.usda.gov/awdbRestApi/services/v1/reference-data?referenceLists=elements')
station_metadata = requests.get('https://wcc.sc.egov.usda.gov/awdbRestApi/services/v1/stations?stationTriplets=784%2CCA%2CSNTL&returnForecastPointMetadata=false&returnReservoirMetadata=false&returnStationElements=true&activeOnly=true')

In [3]:
weather_data_json = weather_data.json()
reference_data_json = reference_data.json()
station_metadata_json = station_metadata.json()

In [4]:
# Step 1: Expand top level -> list of stations
weather_data_list = []
station_metadata_list = []

for station in weather_data_json:
    station_triplet = station["stationTriplet"]
    for measurement in station["data"]:
        element_code = measurement["stationElement"]["elementCode"]
        for val in measurement["values"]:
            weather_data_list.append({
                "stationTriplet": station_triplet,
                "elementCode": element_code,
                "date": val["date"],
                "value": val["value"]
            })

for station in station_metadata_json:
    for element in station["stationElements"]:
        station_metadata_list.append({
            "elementCode": element["elementCode"],
            "durationName": element["durationName"],
            "storedUnitCode": element["storedUnitCode"]
        })

In [5]:
weather_data_df = pl.DataFrame(weather_data_list)
station_metadata_df = pl.DataFrame(station_metadata_list)
reference_data_df = pl.DataFrame(reference_data_json['elements'])

# Every unique string is stored once internally when categorical, and the column is represented as integers
weather_data_df = weather_data_df.with_columns([
    pl.col("stationTriplet").cast(pl.Categorical),
    pl.col("elementCode").cast(pl.Categorical)
])

weather_data_df = weather_data_df.with_columns(
    pl.col("date").str.strptime(pl.Datetime, "%Y-%m-%d %H:%M")
).pivot(values="value", columns="elementCode")


display(weather_data_df)
display(station_metadata_df)
display(reference_data_df)

<positron-console-cell-5>:11: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.


stationTriplet,date,SNWD,TOBS,WTEQ
cat,datetime[μs],i64,i64,i64
"""784:CA:SNTL""",2025-10-01 13:00:00,0,42,null
"""784:CA:SNTL""",2025-10-01 14:00:00,0,43,null
"""784:CA:SNTL""",2025-10-01 15:00:00,0,43,null
"""784:CA:SNTL""",2025-10-01 16:00:00,0,43,null
"""784:CA:SNTL""",2025-10-01 17:00:00,0,42,null
…,…,…,…,…
"""784:CA:SNTL""",2025-12-18 23:00:00,null,34,0
"""784:CA:SNTL""",2025-12-19 02:00:00,null,36,0
"""784:CA:SNTL""",2025-12-19 04:00:00,null,34,0


elementCode,durationName,storedUnitCode
str,str,str
"""BATT""","""DAILY""","""volt"""
"""BATT""","""HOURLY""","""volt"""
"""PRCP""","""CALENDAR_YEAR""","""in"""
"""PRCP""","""DAILY""","""in"""
"""PRCP""","""SEMIMONTHLY""","""in"""
…,…,…
"""WTEQ""","""HOURLY""","""in"""
"""WTEQ""","""MONTHLY""","""in"""
"""WTEQX""","""SEMIMONTHLY""","""in"""


code,name,physicalElementName,functionCode,dataPrecision,description,storedUnitCode,englishUnitCode,metricUnitCode
str,str,str,str,i64,str,str,str,str
"""TAVG""","""AIR TEMPERATURE AVERAGE""","""air temperature""","""V""",1,"""Average Air Temperature - Sub-…","""degF""","""degF""","""degC"""
"""TMAX""","""AIR TEMPERATURE MAXIMUM""","""air temperature""","""X""",1,"""Maximum Air Temperature - Sub-…","""degF""","""degF""","""degC"""
"""TMIN""","""AIR TEMPERATURE MINIMUM""","""air temperature""","""N""",1,"""Minimum Air Temperature - Sub-…","""degF""","""degF""","""degC"""
"""TOBS""","""AIR TEMPERATURE OBSERVED""","""air temperature""","""C""",1,"""Instantaneously Observed Air T…","""degF""","""degF""","""degC"""
"""PRES""","""BAROMETRIC PRESSURE""","""barometric pressure""","""C""",2,"""Barometric Pressure""","""inch_Hg""","""inch_Hg""","""mbar"""
…,…,…,…,…,…,…,…,…
"""WDMVT""","""WIND MOVEMENT TOTAL""","""wind movement""","""S""",0,"""Total Wind Movement""","""mile""","""mile""","""km"""
"""WSPDV""","""WIND SPEED AVERAGE""","""wind speed""","""V""",1,"""Average Wind Speed""","""mph""","""mph""","""km/hr"""
"""WSPDX""","""WIND SPEED MAXIMUM""","""wind speed""","""X""",1,"""Maximum Wind Speed""","""mph""","""mph""","""km/hr"""


In [6]:
# Join Station Metadata with Reference to see what the station reports
available_metrics = station_metadata_df.join(reference_data_df, left_on="elementCode", right_on="code", how="inner").select(['elementCode', 'durationName', 'name', 'description', 'englishUnitCode'])
distinct_metrics = available_metrics['elementCode', 'name', 'description', 'englishUnitCode'].unique()

display(available_metrics)
display(distinct_metrics)

elementCode,durationName,name,description,englishUnitCode
str,str,str,str,str
"""BATT""","""DAILY""","""BATTERY""","""Battery Voltage""","""volt"""
"""BATT""","""HOURLY""","""BATTERY""","""Battery Voltage""","""volt"""
"""PRCP""","""CALENDAR_YEAR""","""PRECIPITATION INCREMENT""","""Total Precipitation""","""in"""
"""PRCP""","""DAILY""","""PRECIPITATION INCREMENT""","""Total Precipitation""","""in"""
"""PRCP""","""SEMIMONTHLY""","""PRECIPITATION INCREMENT""","""Total Precipitation""","""in"""
…,…,…,…,…
"""WTEQ""","""HOURLY""","""SNOW WATER EQUIVALENT""","""Depth of water that would theo…","""in"""
"""WTEQ""","""MONTHLY""","""SNOW WATER EQUIVALENT""","""Depth of water that would theo…","""in"""
"""WTEQX""","""SEMIMONTHLY""","""SNOW WATER EQUIVALENT MAXIMUM""","""Maximum Snow Water Equivalent …","""in"""


elementCode,name,description,englishUnitCode
str,str,str,str
"""STO""","""SOIL TEMPERATURE OBSERVED""","""Observed Soil Temperature ""","""degF"""
"""STV""","""SOIL TEMPERATURE AVERAGE""","""Average Soil Temperature based…","""degF"""
"""STX""","""SOIL TEMPERATURE MAXIMUM""","""Maximum Soil Temperature based…","""degF"""
"""PRCPMTD""","""PRECIPITATION MONTH-TO-DATE""","""Month-to-date Precipitation""","""in"""
"""WSPDX""","""WIND SPEED MAXIMUM""","""Maximum Wind Speed""","""mph"""
…,…,…,…
"""SNWD""","""SNOW DEPTH""","""Total Snow Depth""","""in"""
"""SMV""","""SOIL MOISTURE PERCENT AVERAGE""","""Average Percent Soil Moisture …","""pct"""
"""SMN""","""SOIL MOISTURE PERCENT MINIMUM""","""Minimum Percent Soil Moisture …","""pct"""


In [7]:
# Average snowfall per day


https://demo-seattle-weather.streamlit.app/?ref=streamlit-io-gallery-favorites
# great example of dashboard with weather stats

In [8]:
out = (
    weather_data_df
    .with_columns(
        pl.col("date").dt.truncate("1d") # truncate to year-month-day
    )
    .group_by("date")
    .agg([
        pl.col("TOBS").max().alias("max_value"),
        pl.col("TOBS").min().alias("min_value"),
    ])
)

out

date,max_value,min_value
datetime[μs],i64,i64
2025-10-11 00:00:00,35,28
2025-10-15 00:00:00,37,25
2025-10-16 00:00:00,41,30
2025-10-17 00:00:00,52,34
2025-10-22 00:00:00,51,41
…,…,…
2026-01-03 00:00:00,32,26
2026-01-08 00:00:00,20,11
2026-01-10 00:00:00,39,23


In [9]:
chart = (alt.Chart(out).mark_line().encode(
    x='date',
    y='max_value')
    )

chart

alt.Chart(...)

Visualize SWE bar chart to represent different layers of snow

In [10]:
snow_density_calculation = weather_data_df.filter(pl.col('SNWD').is_not_null() & pl.col('WTEQ').is_not_null()).with_columns(
    (pl.col("WTEQ") / pl.col("SNWD"))
    .alias("snow_density")
).filter(pl.col("snow_density").is_not_nan())
display(snow_density_calculation)

stationTriplet,date,SNWD,TOBS,WTEQ,snow_density
cat,datetime[μs],i64,i64,i64,f64
"""784:CA:SNTL""",2025-10-13 10:00:00,1,29,0,0.0
"""784:CA:SNTL""",2025-10-13 11:00:00,1,30,0,0.0
"""784:CA:SNTL""",2025-10-13 12:00:00,1,30,0,0.0
"""784:CA:SNTL""",2025-10-13 13:00:00,1,30,0,0.0
"""784:CA:SNTL""",2025-10-13 14:00:00,1,30,0,0.0
…,…,…,…,…,…
"""784:CA:SNTL""",2026-01-15 14:00:00,48,46,15,0.3125
"""784:CA:SNTL""",2026-01-15 15:00:00,47,46,15,0.319149
"""784:CA:SNTL""",2026-01-15 16:00:00,48,42,15,0.3125


In [ ]:
# -----------------------------
# Base chart with x-axis encoding
# -----------------------------
base = alt.Chart(weather_data_df).encode(
    x=alt.X(
        "date:T",
        title="",
        axis=alt.Axis(
            grid=False,
            domain=False,
            tickSize=0,
            labelColor="#A8B3C7"
        )
    )
)

# -----------------------------
# Snow depth area chart
# -----------------------------
snow_area = base.mark_area(
    color="lightblue",
    interpolate="step-after",
    line=True
).encode(
    y=alt.Y(
        "SNWD:Q",
        title="Snow Depth (Inches)",
        axis=alt.Axis(
            grid=True,
            gridColor="#EEF2F7",
            domain=False,
            tickSize=0,
            labelColor="#A8B3C7"
        )
    )
)

# -----------------------------
# Render chart
# -----------------------------
chart = snow_area.properties(
    width=700,
    height=380,
    background="#F9FBFD",
    title=alt.TitleParams(
        text="Snow Depth Over Time",
        anchor="start",
        fontSize=16,
        color="#2E3440"
    )
).configure_view(
    stroke=None
).configure_axis(
    labelFontSize=11,
    titleFontSize=12
)

chart

alt.Chart(...)

In [12]:
alt.Chart(weather_data_df).mark_line().encode(
    x='date',
    y='WTEQ')


alt.Chart(...)

In [13]:
alt.Chart(snow_density_calculation).mark_line().encode(
    x='date:T',
    y=alt.Y('snow_density:Q',
            scale=alt.Scale(domain=[0, 1]))  # Don't force to include 0
)


alt.Chart(...)

In [14]:
def daily_temperature_summary(df, datetime_col="date", temp_col="TOBS"):
    return (
        df
        .with_columns(
            pl.col(datetime_col).dt.date().alias("date")
        )
        .group_by("date")
        .agg(
            pl.col(temp_col).min().alias("tmin"),
            pl.col(temp_col).max().alias("tmax"),
            pl.col(temp_col).mean().alias("tavg"),
        )
        .sort("date")
    )

# Example usage
daily_df = daily_temperature_summary(
    weather_data_df,
    datetime_col="date",
    temp_col="TOBS"
)

In [15]:
base = alt.Chart(daily_df).encode(
    x=alt.X(
        "date:T",
        title="",
        axis=alt.Axis(
            grid=False,
            domain=False,
            tickSize=0,
            labelColor="#A8B3C7"
        )
    )
)

# -----------------------------
# Min–Max temperature band
# -----------------------------
band = base.mark_area(
    interpolate="monotone",
    opacity=0.35,
    color="#C7DCEF"
).encode(
    y=alt.Y(
        "tmax:Q",
        title="Temperature",
        axis=alt.Axis(
            grid=True,
            gridColor="#EEF2F7",
            domain=False,
            tickSize=0,
            labelColor="#A8B3C7"
        )
    ),
    y2="tmin:Q"
)

# -----------------------------
# Average temperature line
# -----------------------------
avg_line = base.mark_line(
    color="black",
    strokeWidth=2
).encode(
    y="tavg:Q",
    tooltip=[
        alt.Tooltip("date:T", title="Date"),
        alt.Tooltip("tmin:Q", title="Min Temp"),
        alt.Tooltip("tavg:Q", title="Avg Temp"),
        alt.Tooltip("tmax:Q", title="Max Temp"),
    ]
)

# -----------------------------
# Combine and render
# -----------------------------
chart = (band + avg_line).properties(
    width=700,
    height=380,
    background="#F9FBFD",
    title=alt.TitleParams(
        text="Daily Temperature Range",
        anchor="start",
        fontSize=16,
        color="#2E3440"
    )
).configure_view(
    stroke=None
).configure_axis(
    labelFontSize=11,
    titleFontSize=12
)

chart

alt.LayerChart(...)